In [1]:
#https://www.kaggle.com/errearanhas/topic-modelling-lda-on-elon-tweets/notebook
import os
import numpy as np 
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go

#init_notebook_mode(connected=True) #do not miss this line

from gensim import corpora, models, similarities

import warnings
warnings.filterwarnings("ignore")

C:\Users\Ayah\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning:

detected Windows; aliasing chunkize to chunkize_serial



In [6]:
datafile = 'English_clean_tweet.csv'

In [7]:
import pandas as pd
tweets = pd.read_csv(datafile)

In [8]:
len(tweets)

2425

In [9]:
tweets.head()

,Unnamed: 0,text
0,0,please use the full address in the browser
1,1,please use the full address in the browser
2,2,please try using the full address
3,3,vote casting for the second round of the parli...
4,4,bahraini citizen proudly casting her vote in m...


In [10]:
range(len(tweets['text']))

range(0, 2425)

***Tweet Activity Over Years***

In [11]:
corpus=[]
a=[]
for i in range(len(tweets['text'])):
        a=tweets['text'][i]
        corpus.append(a)

In [12]:
corpus[0:5]

['please use the full address in the browser',
 'please use the full address in the browser',
 'please try using the full address',
 'vote casting for the second round of the parliamentary elections started at the embassy today',
 'bahraini citizen proudly casting her vote in moscow']

In [13]:
import gensim
import logging
import tempfile

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "C:\Users\Ayah\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [15]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [16]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'bahrain.dict'))  # store the dictionary, for future reference
#print(dictionary)

2018-11-03 16:19:57,271 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-11-03 16:19:57,338 : INFO : built Dictionary(4127 unique tokens: ['address', 'browser', 'full', 'please', 'use']...) from 2425 documents (total 25641 corpus positions)
2018-11-03 16:19:57,339 : INFO : saving Dictionary object under C:\Users\Ayah\AppData\Local\Temp\bahrain.dict, separately None
2018-11-03 16:19:57,351 : INFO : saved C:\Users\Ayah\AppData\Local\Temp\bahrain.dict


In [17]:
#print(dictionary.token2id)

In [18]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

2018-11-03 16:20:00,739 : INFO : storing corpus in Matrix Market format to C:\Users\Ayah\AppData\Local\Temp\elon.mm
2018-11-03 16:20:00,742 : INFO : saving sparse matrix to C:\Users\Ayah\AppData\Local\Temp\elon.mm
2018-11-03 16:20:00,743 : INFO : PROGRESS: saving document #0
2018-11-03 16:20:00,777 : INFO : PROGRESS: saving document #1000
2018-11-03 16:20:00,809 : INFO : PROGRESS: saving document #2000
2018-11-03 16:20:00,820 : INFO : saved 2425x4127 matrix, density=0.250% (24978/10007975)
2018-11-03 16:20:00,873 : INFO : saving MmCorpus index to C:\Users\Ayah\AppData\Local\Temp\elon.mm.index


In the previous cells, we created a corpus of documents represented as a stream of vectors. To continue, let’s fire up gensim and use that corpus:

In [19]:
from gensim import corpora, models, similarities

### Creating a transformation


The transformations are standard Python objects, typically initialized by means of a training corpus:

Different transformations may require different initialization parameters; in case of TfIdf, the “training” consists simply of
going through the supplied corpus once and computing document frequencies of all its features.
Training other models, such as Latent Semantic Analysis or Latent Dirichlet Allocation, is much more involved and,
consequently, takes much more time.

In [20]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2018-11-03 16:20:05,124 : INFO : collecting document frequencies
2018-11-03 16:20:05,127 : INFO : PROGRESS: processing document #0
2018-11-03 16:20:05,138 : INFO : calculating IDF weights for 2425 documents and 4126 features (24978 matrix non-zeros)


### Note
Transformations always convert between two specific vector spaces. The same vector space (= the same set of feature ids) must be used for training as well as for subsequent vector transformations. Failure to use the same input feature space, such as applying a different string preprocessing, using different feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will result in feature mismatch during transformation calls and consequently in either garbage output and/or runtime exceptions.

From now on, tfidf is treated as a read-only object that can be used to apply a transformation to a whole corpus:

In [21]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

### LDA:
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

Latent Dirichlet Allocation, LDA is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

In [22]:
total_topics = 5

In [23]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-11-03 16:20:13,477 : INFO : using symmetric alpha at 0.2
2018-11-03 16:20:13,481 : INFO : using symmetric eta at 0.2
2018-11-03 16:20:13,484 : INFO : using serial LDA version on this node
2018-11-03 16:20:13,491 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 2425 documents, updating model once every 2000 documents, evaluating perplexity every 2425 documents, iterating 50x with a convergence threshold of 0.001000
2018-11-03 16:20:13,492 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-11-03 16:20:13,494 : INFO : PROGRESS: pass 0, at document #2000/2425
2018-11-03 16:20:14,517 : INFO : merging changes from 2000 documents into a model of 2425 documents
2018-11-03 16:20:14,517 : INFO : topic #0 (0.200): 0.032*"bahrain" + 0.024*"fm" + 0.024*"kingdom" + 0.015*"foreign" + 0.015*"affairs" + 0.015*"republic" + 0.015*"receives" + 0.011*"ambassador" + 0

In [24]:
#Show first n important word in the topics:
lda.show_topics(total_topics,6)

[(0,
  '0.021*"bahrain" + 0.014*"relevant" + 0.013*"procedures" + 0.013*"taken" + 0.011*"kingdom" + 0.011*"fm"'),
 (1,
  '0.030*"traffic" + 0.023*"police" + 0.016*"salman" + 0.013*"towards" + 0.013*"bin" + 0.013*"shaikh"'),
 (2,
  '0.013*"civil" + 0.013*"defence" + 0.011*"traffic" + 0.011*"security" + 0.011*"pt" + 0.010*"details"'),
 (3,
  '0.039*"interior" + 0.039*"minister" + 0.028*"pt" + 0.028*"security" + 0.022*"receives" + 0.016*"bahrain"'),
 (4,
  '0.021*"bahrain" + 0.014*"minister" + 0.012*"affairs" + 0.010*"king" + 0.010*"receives" + 0.009*"pt"')]

In [25]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [26]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(82, 5)
(5, 82)


In [27]:
df_lda

,accident,affairs,al,alternative,ambassador,amn,anti,arrested,bahrain,bahraini,...,summer,taken,terror,terrorist,towards,traffic,training,us,visits,website
0,0.000000,0.007615,0.000000,0.000000,0.006502,0.0000,0.000000,0.000000,0.020993,0.00000,...,0.000000,0.013009,0.000000,0.005710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.009858,0.000000,0.000000,0.011725,0.000000,0.0000,0.000000,0.007459,0.000000,0.00000,...,0.006952,0.000000,0.000000,0.000000,0.013298,0.030451,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.009475,0.000000,0.000000,0.0072,0.000000,0.000000,0.004815,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.011263,0.004977,0.000000,0.006852,0.005867
3,0.000000,0.005491,0.000000,0.000000,0.011294,0.0000,0.000000,0.000000,0.016139,0.00000,...,0.000000,0.000000,0.004622,0.004022,0.000000,0.000000,0.000000,0.008617,0.000000,0.000000
4,0.000000,0.012060,0.000000,0.000000,0.000000,0.0000,0.005079,0.000000,0.020755,0.00484,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.007822,0.000000,0.000000,0.000000,0.000000


In [29]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics          x          y
topic                                                  
0      43.882263        1       1  60.068279  43.149746
2      25.873808        1       2  12.946766 -66.216545
4      10.196352        1       3   1.553039   1.861586
1      10.027431        1       4  81.723183 -86.182693
3      10.020142        1       5  92.798386 -17.849485, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
63    Default  23.000000     minister  23.000000  30.0000  30.0000
1248  Default  18.000000     interior  18.000000  29.0000  29.0000
2682  Default  18.000000      traffic  18.000000  28.0000  28.0000
492   Default  17.000000     security  17.000000  27.0000  27.0000
2707  Default  22.000000           pt  22.000000  26.0000  26.0000
64    Default  20.000000     receives  20.000000  25.0000  25.0000
1061  Default  18.000000       police  18.000000  24.0000  24.0000
23    Default  35.000000      bahrain  35.000000  23.0000  23.0000
66    Default  10.000000   ambassador  10.000000  22.0000  22.0000
516   Default   4.000000       salman   4.000000  21.0000  21.0000
1088  Default   7.000000       public   7.000000  20.0000  20.0000
379   Default   6.000000      towards   6.000000  19.0000  19.0000
55    Default  13.000000      affairs  13.000000  18.0000  18.0000
975   Default   5.000000        guard   5.000000  17.0000  17.0000
120   Default   4.000000          bin   4.000000  16.0000  16.0000
269   Default   3.000000       shaikh   3.000000  15.0000  15.0000
1946  Default   3.000000        roads   3.000000  14.0000  14.0000
2791  Default   4.000000      highway   4.000000  13.0000  13.0000
1472  Default   3.000000  alternative   3.000000  12.0000  12.0000
2011  Default   3.000000         seek   3.000000  11.0000  11.0000
374   Default   7.000000         king   7.000000  10.0000  10.0000
268   Default   3.000000      khalifa   3.000000   9.0000   9.0000
1573  Default   3.000000       kindly   3.000000   8.0000   8.0000
466   Default  16.000000     relevant  16.000000   7.0000   7.0000
2612  Default  15.000000   procedures  15.000000   6.0000   6.0000
146   Default   4.000000        chief   4.000000   5.0000   5.0000
1445  Default   2.000000        heavy   2.000000   4.0000   4.0000
918   Default   6.000000         near   6.000000   3.0000   3.0000
990   Default   3.000000     accident   3.000000   2.0000   2.0000
848   Default   3.000000        coast   3.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
1088   Topic5   3.711873       public   7.346732   1.6179  -4.1815
637    Topic5   0.959203          hrh   1.643844   1.7619  -5.5347
63     Topic5   9.529872     minister  23.043964   1.4176  -3.2386
492    Topic5   6.798670     security  17.702009   1.3436  -3.5763
227    Topic5   2.124339       deputy   4.637144   1.5199  -4.7396
725    Topic5   0.715930       prince   1.273124   1.7249  -5.8272
2707   Topic5   6.866441           pt  22.752100   1.1026  -3.5664
1947   Topic5   1.123052       terror   2.258588   1.6019  -5.3770
64     Topic5   5.247801     receives  20.375414   0.9441  -3.8352
66     Topic5   2.744196   ambassador  10.170860   0.9905  -4.4835
28     Topic5   2.093784           us   8.425203   0.9083  -4.7540
23     Topic5   3.921696      bahrain  35.225159   0.1053  -4.1265
616    Topic5   1.006711           hm   3.114157   1.1713  -5.4863
784    Topic5   0.489565           hh   0.864628   1.7318  -6.2073
374    Topic5   1.481582         king   7.984470   0.6162  -5.0999
950    Topic5   0.720395        visit   1.931212   1.3145  -5.8210
39     Topic5   0.835785          new   2.706501   1.1255  -5.6724
502    Topic5   1.162200         iran   5.515458   0.7433  -5.3427
88     Topic5   1.351660     ministry   7.727928   0.5571  -5.1917
581    Topic5   0.534481       qatari   1.045503   1.6296  -6.1195
148    Topic5 

In [30]:
lda.print_topics(num_topics=5)

2018-11-03 16:21:30,376 : INFO : topic #0 (0.200): 0.021*"bahrain" + 0.014*"relevant" + 0.013*"procedures" + 0.013*"taken" + 0.011*"kingdom" + 0.011*"fm" + 0.010*"receives" + 0.010*"fire" + 0.009*"defence" + 0.008*"civil"
2018-11-03 16:21:30,379 : INFO : topic #1 (0.200): 0.030*"traffic" + 0.023*"police" + 0.016*"salman" + 0.013*"towards" + 0.013*"bin" + 0.013*"shaikh" + 0.013*"guard" + 0.012*"highway" + 0.012*"roads" + 0.012*"alternative"
2018-11-03 16:21:30,381 : INFO : topic #2 (0.200): 0.013*"civil" + 0.013*"defence" + 0.011*"traffic" + 0.011*"security" + 0.011*"pt" + 0.010*"details" + 0.010*"held" + 0.009*"al" + 0.008*"fire" + 0.007*"amn"
2018-11-03 16:21:30,382 : INFO : topic #3 (0.200): 0.039*"interior" + 0.039*"minister" + 0.028*"pt" + 0.028*"security" + 0.022*"receives" + 0.016*"bahrain" + 0.015*"public" + 0.011*"ambassador" + 0.009*"chief" + 0.009*"deputy"
2018-11-03 16:21:30,385 : INFO : topic #4 (0.200): 0.021*"bahrain" + 0.014*"minister" + 0.012*"affairs" + 0.010*"king" + 

[(0,
  '0.021*"bahrain" + 0.014*"relevant" + 0.013*"procedures" + 0.013*"taken" + 0.011*"kingdom" + 0.011*"fm" + 0.010*"receives" + 0.010*"fire" + 0.009*"defence" + 0.008*"civil"'),
 (1,
  '0.030*"traffic" + 0.023*"police" + 0.016*"salman" + 0.013*"towards" + 0.013*"bin" + 0.013*"shaikh" + 0.013*"guard" + 0.012*"highway" + 0.012*"roads" + 0.012*"alternative"'),
 (2,
  '0.013*"civil" + 0.013*"defence" + 0.011*"traffic" + 0.011*"security" + 0.011*"pt" + 0.010*"details" + 0.010*"held" + 0.009*"al" + 0.008*"fire" + 0.007*"amn"'),
 (3,
  '0.039*"interior" + 0.039*"minister" + 0.028*"pt" + 0.028*"security" + 0.022*"receives" + 0.016*"bahrain" + 0.015*"public" + 0.011*"ambassador" + 0.009*"chief" + 0.009*"deputy"'),
 (4,
  '0.021*"bahrain" + 0.014*"minister" + 0.012*"affairs" + 0.010*"king" + 0.010*"receives" + 0.009*"pt" + 0.008*"police" + 0.008*"traffic" + 0.007*"kingdom" + 0.007*"interior"')]